In [45]:
from cng.utils import *
from cng.h3 import *
from ibis import _
import ibis.selectors as s
import os
from osgeo import gdal
from minio import Minio
import streamlit as st
from datetime import timedelta
import geopandas as gpd
import re
import pandas as pd
# Get signed URLs to access license-controlled layers
key = st.secrets["MINIO_KEY"]
secret = st.secrets["MINIO_SECRET"]
client = Minio("minio.carlboettiger.info", key, secret)

con = ibis.duckdb.connect(extensions = ["spatial", "h3"])
endpoint = os.getenv("AWS_S3_ENDPOINT", "minio.carlboettiger.info")
duckdb_install_h3()

set_secrets(con)

In [46]:
# All U.S. state abbreviations
state_ids = [
    "AL","AK","AZ","AR","CA","CO","CT","DE","FL","GA","HI","ID","IL","IN","IA",
    "KS","KY","LA","ME","MD","MA","MI","MN","MS","MO","MT","NE","NV","NH","NJ",
    "NM","NY","NC","ND","OH","OK","OR","PA","RI","SC","SD","TN","TX","UT","VT",
    "VA","WA","WV","WI","WY"
]

In [51]:
import pandas as pd

implementation_sources = [
    {
        "state_id": "CA",
        "implementation_id": "CA_EO_N82_20",
        "implementation_type": "Executive Order",
        "implementation_notes": "Establishes CA’s 30×30 conservation goal and directs implementation.",
        "implementation_citation": "State of California. (2020, October 7). Executive Order N-82-20.",
        "implementation_url": "https://www.gov.ca.gov/wp-content/uploads/2020/10/10.07.2020-EO-N-82-20-.pdf",
    },
    {
        "state_id": "CA",
        "implementation_id": "CA_SB_337",
        "implementation_type": "Statute",
        "implementation_notes": "Codifies 30×30 conservation goal in statute (chaptered 2023).",
        "implementation_citation": "California Legislature. (2023). Senate Bill 337 (Chapter 392, Statutes of 2023): Environmental protection: lands and coastal waters conservation goal. Approved October 7, 2023.",
        "implementation_url": "https://leginfo.legislature.ca.gov/faces/billTextClient.xhtml?bill_id=202320240SB337"
    },
    {
        "state_id": "HI",
        "implementation_id": "HI_HOLOMUA_2020",
        "implementation_type": "Executive Commitment",
        "implementation_notes": "DLNR marine 30×30 roadmap (program guidance; not a statute/EO)",
        "implementation_citation": "Hawaiʻi Department of Land and Natural Resources, Division of Aquatic Resources. (2020, December). Holomua: Marine 30×30 — Roadmap to effective management of near-shore waters.",
        "implementation_url": "https://dlnr.hawaii.gov/dar/files/2020/12/HolomuaMarine30x30_Roadmap_final.pdf",
    },
    {
        "state_id": "IL",
        "implementation_id": "IL_PA_102_0618",
        "implementation_type": "Statute (Other)",
        "implementation_notes": "Creates 30×30 task force (study/recommendations); no binding statewide 30×30 mandate.",
        "implementation_citation": "State of Illinois. (2021, August 27). Public Act 102-0618: Illinois Thirty-by-Thirty Conservation Task Force Act.",
        "implementation_url": "https://www.ilga.gov/Legislation/publicacts/view/102-0618",
    },
    {
        "state_id": "IL",
        "implementation_id": "IL_SR0632_103GA",
        "implementation_type": "Legislative Resolution",
        "implementation_notes": "Senate Resolution 632 – Establishing a 30x30 Coordinating Council; non-binding legislative expression of support.",
        "implementation_citation": "Illinois General Assembly. (2024). Senate Resolution 632 – Establishing a 30x30 Coordinating Council [Resolution].",
        "implementation_url": "https://www.ilga.gov/Legislation/BillStatus?DocNum=632&GAID=17&DocTypeID=SR&LegId=151047&SessionID=112&GA=103",
    },
    {
        "state_id": "NV",
        "implementation_id": "NV_AJR_3_2021",
        "implementation_type": "Legislative Resolution",
        "implementation_notes": "AJR No. 3 (81st Leg., 2021); non-binding legislative expression of support.",
        "implementation_citation": "Nevada Legislature. (2021). Assembly Joint Resolution No. 3 (81st Leg., 2021).",
        "implementation_url": "https://www.leg.state.nv.us/App/NELIS/REL/81st2021/Bill/7487/Overview",
    },
    {
        "state_id": "ME",
        "implementation_id": "ME_MAINE_WONT_WAIT",
        "implementation_type": "Executive Commitment",
        "implementation_notes": "State climate action plan that includes Maine’s 30×30 conservation goal.",
        "implementation_citation": "State of Maine. (2020, December 1). Maine Won’t Wait: A Four‑Year Plan for Climate Action.",
        "implementation_url": "https://www.maine.gov/future/sites/maine.gov.future/files/inline-files/MaineWontWait_December2020.pdf",
    },
    {
        "state_id": "MA",
        "implementation_id": "MA_EO_618",
        "implementation_type": "Executive Commitment",
        "implementation_notes": "Executive order establishing biodiversity conservation framework aligned with 30×30 objectives.",
        "implementation_citation": "Commonwealth of Massachusetts. (2023, September 21). Executive Order No. 618: Biodiversity conservation in Massachusetts.",
        "implementation_url": "https://www.mass.gov/executive-orders/no-618-biodiversity-conservation-in-massachusetts",
    },
    {
        "state_id": "MD",
        "implementation_id": "MD_SB_470",
        "implementation_type": "Statute",
        "implementation_notes": "Statute codifying Maryland’s 30×30 conservation goal and implementation framework.",
        "implementation_citation": "State of Maryland. (2023). Senate Bill 470: Maryland the Beautiful Act (Ch. 546).",
        "implementation_url": "https://www.mgaleg.maryland.gov/2023RS/chapters_noln/Ch_546_sb0470E.pdf",
    },
    {
        "state_id": "MI",
        "implementation_id": "MI_ATB_FRESHWATER_CHALLENGE",
        "implementation_type": "Executive Commitment",
        "implementation_notes": "Gov. Whitmer commits to freshwater conservation under the America the Beautiful Freshwater Challenge.",
        "implementation_citation": "State of Michigan, Office of the Governor. (2024, June 27). Gov. Whitmer accepts invitation to join White House initiative to conserve and restore freshwater resources: America the Beautiful Freshwater Challenge.",
        "implementation_url": "https://www.michigan.gov/whitmer/news/press-releases/2024/06/27/gov-whitmer-accepts-invitation-to-join-white-house-initiative",
    },
    {
        "state_id": "MI",
        "implementation_id": "MI_SB_1124",
        "implementation_type": "Proposed Legislation",
        "implementation_notes": "Proposed statute to establish a 30×30 protection goal.",
        "implementation_citation": "Michigan Legislature. (2024). Senate Bill 1124.",
        "implementation_url": "https://www.legislature.mi.gov/Bills/Bill?ObjectName=2024‑SB‑1124",
    },
    {
        "state_id": "NJ",
        "implementation_id": "NJ_Joins_HACforN&P",
        "implementation_type": "Executive Commitment",
        "implementation_notes": "Gov. Murphy joins the High Ambition Coalition for Nature and People (30×30 coalition).",
        "implementation_citation": "State of New Jersey, Office of the Governor. (2025, June 3). New Jersey joins Global Commitment to Preserve 30 % of Natural Lands by 2030.",
        "implementation_url": "https://www.nj.gov/governor/news/news/562025/approved/20250603c.shtml",
    },
    {
        "state_id": "NM",
        "implementation_id": "NM_EO_2021_052",
        "implementation_type": "Executive Order",
        "implementation_notes": "Executive order establishing New Mexico’s 30×30 conservation framework.",
        "implementation_citation": "State of New Mexico. (2021, August 25). Executive Order 2021‑052: Protecting New Mexico’s Lands, Watersheds, Wildlife, and Natural Heritage.",
        "implementation_url": "https://www.governor.state.nm.us/wp-content/uploads/2021/08/Executive-Order-2021-052.pdf",
    },
    {
        "state_id": "NY",
        "implementation_id": "NY_ECL_49_0113",
        "implementation_type": "Statute",
        "implementation_notes": "Statute codifying New York’s 30×30 conservation goal.",
        "implementation_citation": "State of New York. (2022, December 23). Environmental Conservation Law § 49‑0113 — Thirty by thirty conservation goal.",
        "implementation_url": "https://www.nysenate.gov/legislation/laws/ENV/49‑0113",
    },
    {
        "state_id": "SC",
        "implementation_id": "SC_S1024_2020",
        "implementation_type": "Proposed Legislation",
        "implementation_notes": "Proposed statute to establish a 30×30 protection goal and corresponding interagency task force.",
        "implementation_citation": "South Carolina Legislature. (2020). S.1024: Environmental protection and conservation (123rd Leg.)",
        "implementation_url": "https://www.scstatehouse.gov/sess123_2019-2020/bills/1024.htm",
    },
    {
        "state_id": "SC",
        "implementation_id": "SC_S220_2021",
        "implementation_type": "Proposed Legislation",
        "implementation_notes":  "Reintroduced conservation statute in the 124th General Assembly to establish 30x30 goal and interagency taskforce.",
        "implementation_citation": "South Carolina Legislature. (2021). S.220: SC Thirty‑by‑Thirty Conservation Act (124th Leg.)",
        "implementation_url": "https://www.scstatehouse.gov/sess124_2021-2022/bills/220.htm",
    },
    {
        "state_id": "VT",
        "implementation_id": "VT_Act_59",
        "implementation_type": "Statute",
        "implementation_notes": "Statute establishing Vermont’s 30×30 conservation goal and implementation framework.",
        "implementation_citation": "State of Vermont. (2023, June). Community Resilience and Biodiversity Protection Act (Act 59). Vermont General Assembly.",
        "implementation_url": "https://legislature.vermont.gov/Documents/2024/Docs/ACTS/ACT059/ACT059%20As%20Enacted.pdf",
    },
]


In [52]:
progress_sources = [
    {
        "state_id": "CA",
        "progress_type": "active",
        "progress_notes": "Annual report on state progress toward the 30×30 conservation goal.",
        "progress_citation": "California Natural Resources Agency. (2025). Pathways to 30×30 Annual Progress Report 2025.",
        "progress_url": "https://resources.ca.gov/-/media/CNRA-Website/Files/2025-30x30-Pathways-Progress-Report.pdf",
    },
    {
        "state_id": "HI",
        "progress_type": "halted",
        "progress_notes": "DLNR document describing reframing of the initiative and dropping the explicit 30×30 target language.",
        "progress_citation": "Hawaiʻi Department of Land and Natural Resources, Division of Aquatic Resources. (2024, July). Holomua Marine Initiative: A guide to navigating nearshore resource management.",
        "progress_url": "https://dlnr.hawaii.gov/holomua/files/2024/07/Holomua-Guide-Final-July2024.pdf",
    },
     {
        "state_id": "IL",
        "progress_type": "active",
        "progress_notes": "Illinois DNR program page describing the state’s 30 by 30 work and coordinating activities.",
        "progress_citation": "Illinois Department of Natural Resources. (n.d.). Illinois 30 by 30 Conservation Task Force.",
        "progress_url": "https://dnr.illinois.gov/programs/30by30.html",
    },
    {
        "state_id": "NV",
        "progress_type": "active",
        "progress_notes": "",
        "progress_citation": "",
        "progress_url": "",
    },
    {
        "state_id": "ME",
        "progress_type": "active",
        "progress_notes": "",
        "progress_citation": "",
        "progress_url": "",
    },
    {
        "state_id": "MA",
        "progress_type": "active",
        "progress_notes": "Commonwealth page describing biodiversity conservation goals and implementation context (post-EO 618).",
        "progress_citation": "Commonwealth of Massachusetts. (2025). Biodiversity Goals for Massachusetts.",
        "progress_url": "https://www.mass.gov/info-details/biodiversity-goals-for-massachusetts",
    },
    {
        "state_id": "MD",
        "progress_type": "achieved",
        "progress_notes": "State announcement reporting Maryland has met the 30% protected lands milestone ahead of schedule.",
        "progress_citation": "Maryland Department of Natural Resources. (2024, May 15). Governor Moore Announces Maryland Meets 30% of State Land Conservation Goal Six Years Ahead of Schedule.",
        "progress_url": "https://news.maryland.gov/dnr/2024/05/15/governor-moore-announces-maryland-meets-30-of-state-land-conservation-goal-six-years-ahead-of-schedule/",
    },
    {
        "state_id": "MI",
        "progress_type": "active",
        "progress_notes": "",
        "progress_citation": "",
        "progress_url": "",
    },
    {
        "state_id": "NJ",
        "progress_type": "achieved",
        "progress_notes": "State press release includes statement that New Jersey has already achieved the 30×30 goal.",
        "progress_citation": "State of New Jersey, Office of the Governor. (2025, June 3). New Jersey joins Global Commitment to Preserve 30% of Natural Lands by 2030.",
        "progress_url": "https://www.nj.gov/governor/news/news/562025/approved/20250603c.shtml",
    },
        {
        "state_id": "NM",
        "progress_type": "achieved",
        "progress_notes": "Official 30×30 annual report identifies over 25 million acres meeting the state committee’s definition of conserved lands (which is over 30% of NM's area) but does not explicitly declare that New Mexico has achieved the 30×30 goal.",
        "progress_citation": "New Mexico Energy, Minerals and Natural Resources Department, Forestry Division. (2024). 30 × 30 New Mexico: 2024 Annual Report.",
        "progress_url": "https://www.emnrd.nm.gov/sfd/wp-content/uploads/sites/4/30x30-2024-Annual-Report-Final.pdf",
    {
        "state_id": "NY",
        "progress_type": "active",
        "progress_notes": "",
        "progress_citation": "",
        "progress_url": "",
    },
    {
        "state_id": "SC",
        "progress_type": "failed_legislation",
        "progress_notes": "Introduced legislation related to 30×30; Did not pass",
        "progress_citation": "South Carolina Legislature. (2020). S.1024: Environmental protection and conservation (123rd Leg.).",
        "progress_url": "https://www.scstatehouse.gov/sess123_2019-2020/bills/1024.htm",
    },
    {
        "state_id": "SC",
        "progress_type": "failed_legislation",
        "progress_notes": "Reintroduced legislation related to 30×30; Did not pass",
        "progress_citation": "South Carolina Legislature. (2021). S.220: SC Thirty‑by‑Thirty Conservation Act (124th Leg.)",
        "progress_url": "https://www.scstatehouse.gov/sess124_2021-2022/bills/220.htm",
    },
    {
        "state_id": "VT",
        "progress_type": "active",
        "progress_notes": "",
        "progress_citation": "",
        "progress_url": "",
    },
    # ...
]
import pandas as pd

df_sources = pd.concat(
    [pd.DataFrame(implementation_sources), pd.DataFrame(progress_sources)],
    ignore_index=True
)

# Optional: standardize ordering
df_states = pd.DataFrame({"state_id": state_ids})

df_big = df_states.merge(
    df_sources,
    on="state_id",
    how="left",
    validate="1:m"
)


,state_id,implementation_id,implementation_type,implementation_notes,implementation_citation,implementation_url,progress_type,progress_notes,progress_citation,progress_url
0,AL,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,AK,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,AZ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,AR,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,CA,CA_EO_N82_20,Executive Order,Establishes CA’s 30×30 conservation goal and d...,"State of California. (2020, October 7). Execut...",https://www.gov.ca.gov/wp-content/uploads/2020...,NaN,NaN,NaN,NaN
5,CA,CA_SB_337,Statute,Codifies 30×30 conservation goal in statute (c...,California Legislature. (2023). Senate Bill 33...,https://leginfo.legislature.ca.gov/faces/billT...,NaN,NaN,NaN,NaN
6,CA,NaN,NaN,NaN,NaN,NaN,active,Annual report on state progress toward the 30×...,California Natural Resources Agency. (2025). P...,https://resources.ca.gov/-/media/CNRA-Website/...
7,CO,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,CT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,DE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# 30x30 initiative type

# adding polygon boundaries using census data

In [ ]:
url = 's3://public-census/2024/state/2024_us_state.parquet'
data = con.read_parquet(url).select('state_id','geom').execute()

merged_df = df.merge(data, on="state_id", how="left")
merged_df = merged_df.set_geometry('geom')



In [ ]:
# checkerboarding states where it has two values 
import geopandas as gpd
from shapely.geometry import box, Polygon, MultiPolygon
from shapely.prepared import prep

def polygon_checkerboard_two_entries(poly, n_rows=10, n_cols=10):
    """
    Robust checkerboard split for a polygon.
    Returns two geometries: normal and inverse.
    """
    if poly.is_empty:
        return None, None

    minx, miny, maxx, maxy = poly.bounds
    width = (maxx - minx) / n_cols
    height = (maxy - miny) / n_rows

    # Skip if width or height is zero
    if width == 0 or height == 0:
        return None, None

    tiles_0 = []
    tiles_1 = []

    prep_poly = prep(poly)  # speed up intersection checks

    for i in range(n_cols):
        for j in range(n_rows):
            x1, y1 = minx + i*width, miny + j*height
            x2, y2 = minx + (i+1)*width, miny + (j+1)*height

            # Skip degenerate rectangles
            if x1 == x2 or y1 == y2:
                continue

            square = box(x1, y1, x2, y2)

            # Quick check before full intersection
            if not prep_poly.intersects(square):
                continue

            inter = poly.intersection(square)

            # Only append valid polygons
            if inter.is_empty:
                continue
            if isinstance(inter, (Polygon, MultiPolygon)):
                if (i + j) % 2 == 0:
                    tiles_0.append(inter)
                else:
                    tiles_1.append(inter)
            elif hasattr(inter, "geoms"):  # GeometryCollection
                polys = [g for g in inter.geoms if isinstance(g, (Polygon, MultiPolygon))]
                if (i + j) % 2 == 0:
                    tiles_0.extend(polys)
                else:
                    tiles_1.extend(polys)

    geom_0 = gpd.GeoSeries(tiles_0).unary_union if tiles_0 else None
    geom_1 = gpd.GeoSeries(tiles_1).unary_union if tiles_1 else None

    return geom_0, geom_1



# Define states and custom goals
checkerboard_states = {
    "MI": ["ec", "failed"],
    "CA": ["eo", "law"]
}

checkerboard_entries = []

for state, goals in checkerboard_states.items():
    # Get the unioned polygon for the state
    poly = merged_df.loc[merged_df["state_id"]==state, "geom"].unary_union
    
    # Use the function to generate checkerboard polygons
    geom_normal, geom_inverse = polygon_checkerboard_two_entries(poly, n_rows=10, n_cols=10)
    
    # Assign the custom goals
    for g, geom in zip(goals, [geom_normal, geom_inverse]):
        checkerboard_entries.append({
            "state_id": state,
            "goal": g,
            "geom": geom
        })

# Remove original states from merged_df
merged_df = merged_df[~merged_df["state_id"].isin(checkerboard_states.keys())]

# Append the new checkerboard entries
merged_df = pd.concat(
    [merged_df, gpd.GeoDataFrame(checkerboard_entries, geometry="geom", crs="EPSG:4326")],
    ignore_index=True
)

# Ensure GeoDataFrame
merged_df = gpd.GeoDataFrame(merged_df, geometry="geom", crs="EPSG:4326")

# mapping data

In [ ]:
import leafmap.maplibregl as leafmap
m = leafmap.Map(style="positron")

eo_color = "#918EF4"
ec_color = "#42BFDD"
law_color = "#568046"
failed_color = "#E30202"
resolution_color = "#F28C28"
other_color = "#B66D0D"

paint = {
    "fill-color": [
        "match",
        ["get", "goal"],        # the attribute in your GeoDataFrame
        "eo", eo_color,     # color for 30x30
        "ec", ec_color,
        "law", law_color,
        "failed", failed_color,
        "resolution", resolution_color,
        "other", other_color,     # color for other
        "#FFFFFF"               # default for none/anything else
    ],
    "fill-opacity": 0.9
}
m.add_gdf(merged_df, paint = paint, layer_type = 'fill')

legend_dict = {
    "Statute": law_color,
    "Statute (Other)": other_color,
    "Executive Order": eo_color, 
    "Executive Commitment": ec_color,
    "Legislative Resolution": resolution_color, 
    "Proposed Bill": failed_color,
}

m.add_legend(
    title="",
    legend_dict=legend_dict,
    # position='top-right'
)

# Add a boundary line layer
m.add_gdf(
    merged_df,
    layer_type="line",
    paint={
        "line-color": "#000000",   # black borders
        "line-width": 0.8
    },
    name="borders"
)

m